# Set team permissions for all GitHub organisation repositories

This Jupyter notebook illustrates how to use [PyGithub][pygithub], and the [GitHub REST APIv3][github-api] to set a team's permissions for all GitHub organisation repositories.

> ❗️This is a **destructive** action — be very careful before you do this, and make sure you know what you are doing! To prevent accidental execution, the relevant code block is commented out.

[github-api]: https://docs.github.com/en/free-pro-team@latest/rest
[pygithub]: https://pygithub.readthedocs.io/

In [ ]:
from github import Github
from src import extract_attribute_from_dict_of_paginated_lists, find_organisation_repos, get_items_for_all_repos
import os

## Getting started

We first need to load environment variables, instantiate a `github.Github` class, and get all repositories within a Github organisation. Further details can be found in the ["Get GitHub organisation repositories" example Jupyter notebook][example-organisation_repositories].

[example-organisation_repositories]: ./example_get_organisation_repositories.ipynb

In [ ]:
# Load required environment variables
GITHUB_ORGANISATION = os.getenv("GITHUB_ORGANISATION")

# Instantiate a `github.Github` object to gain access to GitHub REST APIv3
g = Github(os.getenv("GITHUB_API_KEY"), per_page=100)

# Get all the repositories for `GITHUB_ORGANISATION`
organisation_repositories = find_organisation_repos(g, GITHUB_ORGANISATION)

## Getting the GitHub organisation team

You also need to define a `github.Team.Team` object for the GitHub organisation team you wish to use. You can use the following code to do this using the `GITHUB_TEAM_SLUG` environment variable. Alternatively, replace:

```python
os.getenv("GITHUB_TEAM_SLUG")
```

with a string of your team name.

In [ ]:
# Get a `github.Team.Team` object for the GitHub organisation team you wish to use
g_team = g.get_organization(GITHUB_ORGANISATION).get_team_by_slug(os.getenv("GITHUB_TEAM_SLUG"))

## Setting a team's permissions for all GitHub organisation repositories

> ⚠️ You can only set permissions for repositories where you already have organisation owner or repository administrative permissions.

The wrapper function `add_team_with_permissions_to_all_repositories` allows you to programmatically:

1. Add a team to a GitHub organisation repository if it isn't added already; and
2. Set the team's permission level.

> ⚠️ Point 1 is important as it prevents overwriting of existing team permissions as, by default, PyGithub will overwite existing team permissions in a repository to the organisation default before you can set a new permission level; in some cases this means the team will be downgraded! For example, if you have admin privileges via a team you will lose them, and then not be able to reset these privileges without assistance from an admin or owner!

The function has three mandatory arguments:

1. The GitHub organisation team to be added as a `github.Team.Team` object;
2. [The permission level required as string][github-team-permissions]; and
3. A list or `github.PaginatedList.PaginatedList` object of `github.Repository.Repository` objects for the team to be added to.

The permission level is defined as in the [GitHub REST APIv3 documentation][github-team-permissions]. The function returns nothing, but you should see the effect of it's actions on GitHub itself.

In our example, we will add `g_team` to all GitHub organisation repositories if not already added, and then set the team to have administrative privileges across all repositories.

[github-team-permissions]: https://docs.github.com/en/free-pro-team@latest/rest/reference/teams#add-or-update-team-project-permissions

In [ ]:
# # Amend `g_team` permissions to admin in each of `organisation_repositories`
# add_team_with_permissions_to_all_repositories(g_team, "admin", organisation_repositories)

### Setting a team's permissions for a specific list of repositories

As mentioned, `add_team_with_permissions_to_all_repositories` takes three mandatory arguments. The third argument can also be a list of `github.Repository.Repository` objects.

This means you can take a subset of `organisation_repositories`, or even filter `organisation_repositories` so you only set a team's permissions for a specific list of repositories.

Let's add `g_team` to the first GitHub organisation repository in `organisation_repositories` if it is not already added, and then set the team to have administrative privileges.

In [ ]:
# # Amend `g_team` permissions to admin in the first repository of `organisation_repositories`
# add_team_with_permissions_to_all_repositories(g_team, "admin", [organisation_repositories[0]])

#### Setting a team's permission for a specific repository name

To filter for a specific repository name, use:

```python
add_team_with_permissions_to_all_repositories(
    g_team, "admin", [r for r in organisation_repositories if r.name == "<<<REPOSITORY_NAME>>>"]
)
```

where `<<<REPOSITORY_NAME>>>` is the repository of interest.

#### Setting a team's permission for a list of specific repository names

For multiple, specificed repository names, use:

```python
add_team_with_permissions_to_all_repositories(
    g_team, "admin",
    [r for r in organisation_repositories if r.name in <<<REPOSITORY_NAME_LIST>>]
)
```

where `<<<REPOSITORY_NAME_LIST>>` is a list of repository names as strings.